In [1]:
import os
import gc
import json
import pathlib
from datetime import datetime
import optuna
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import roc_auc_score
import torch
from typing import List, Dict, Union, Tuple, NamedTuple
from tqdm import tqdm
import scml
from scml import pandasx as pdx
tim = scml.Timer()
tim.start()
os.environ["TOKENIZERS_PARALLELISM"] = "false"
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()

In [2]:
ts = datetime.now().strftime('%Y%m%d_%H%M%S')
job_dir = f"models/xgb/{ts}"
pathlib.Path(job_dir).mkdir(parents=True, exist_ok=True)
num_boost_round: int = 100
lr: Tuple[float, float] = (1e-3, 1e-3)
feature_fraction: Tuple[float, float] = (1, 1)
min_data_in_leaf: Tuple[int, int] = (20, 20)
objective: str = "binary:logistic"
n_trials: int = 1
label = "generated"

In [3]:
df = pd.read_parquet("input/features.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221612 entries, 0 to 221611
Columns: 50047 entries, row_id to tf_Ġyoutube_Ġuploaded_Ġby
dtypes: float32(50033), int16(2), int32(5), int8(1), object(6)
memory usage: 41.3+ GB


In [4]:
features = []
prefixes = ["ch_", "ws_", "ts_", "va_", "tf_"]
for col in df.columns:
    for prefix in prefixes:
        if col.startswith(prefix):
            features.append(col)
features.sort()
print(f"{len(features)} features\n{features[:100]}")

50036 features
['ch_digit_frac', 'ch_len', 'ch_letter_frac', 'ch_punc_frac', 'ch_repeat_char_frac', 'ch_space_frac', 'ch_upper_frac', 'tf_1_n_1', 'tf_a_Ġet_Ġal', 'tf_a_Ġis_Ġa', 'tf_ability_Ġof_Ġthe', 'tf_ality_Ġof_Ġthe', 'tf_an_Ġet_Ġal', 'tf_ance_Ġof_Ġthe', 'tf_ar_Ġet_Ġal', 'tf_as_Ġet_Ġal', 'tf_ated_Ġby_Ġthe', 'tf_ated_Ġin_Ġthe', 'tf_ation_Ġof_Ġthe', 'tf_ations_Ġof_Ġthe', 'tf_e_Ġet_Ġal', 'tf_ed_Ġat_Ġthe', 'tf_ed_Ġby_Ġthe', 'tf_ed_Ġfrom_Ġthe', 'tf_ed_Ġin_Ġthe', 'tf_ed_Ġinto_Ġthe', 'tf_ed_Ġon_Ġthe', 'tf_ed_Ġto_Ġthe', 'tf_ed_Ġwith_Ġthe', 'tf_el_Ġet_Ġal', 'tf_en_Ġet_Ġal', 'tf_er_Ġet_Ġal', 'tf_er_Ġin_Ġthe', 'tf_ers_Ġin_Ġthe', 'tf_ers_Ġof_Ġthe', 'tf_es_Ġet_Ġal', 'tf_es_Ġin_Ġthe', 'tf_es_Ġof_Ġthe', 'tf_i_Ġet_Ġal', 'tf_ia_Ġet_Ġal', 'tf_ification_Ġof_Ġthe', 'tf_in_Ġet_Ġal', 'tf_in_Ġin_Ġthe', 'tf_ing_Ġin_Ġthe', 'tf_ing_Ġof_Ġthe', 'tf_ing_Ġto_Ġthe', 'tf_is_Ġet_Ġal', 'tf_isation_Ġof_Ġthe', 'tf_ism_Ġand_Ġthe', 'tf_ism_Ġin_Ġthe', 'tf_ists_Ġin_Ġthe', 'tf_ities_Ġof_Ġthe', 'tf_ity_Ġof_Ġthe', 'tf_ivenes

In [5]:
#X_train, X_test, y_train, y_test = train_test_split(tra[features], tra[label], test_size=0.2)

th = 0.29
tra = df[df["white_sim"]>=th].copy()
val = df[df["white_sim"]<th].copy()
t = len(tra)
v = len(val)
n = t+v
print(f"val%={v/n:.4f}, len(tra)={t:,}, len(val)={v:,}")
del df
gc.collect()
dtrain = xgb.DMatrix(tra[features], tra[label], enable_categorical=False)
dval = xgb.DMatrix(val[features], val[label], enable_categorical=False)
pdx.value_counts(val[label])

val%=0.0473, len(tra)=211,120, len(val)=10,492


,count,percent
generated,,
0,10451,0.996092
1,41,0.003908


In [6]:
%%time
model = xgb.train(
   params={
       "objective": objective,
       "learning_rate": 0.2,
       "min_child_weight": 20,
       "colsample_bytree": 0.5,
       "max_depth": 6,
   },
   dtrain=dtrain,
   num_boost_round=2000,
   evals=[(dtrain, "train"), (dval, "val")],
   verbose_eval=40,
   early_stopping_rounds=100,
)
print(f"best score {model.best_score:.5f} at iteration {model.best_iteration}")
model.save_model(f"{job_dir}/model.json")

[0]	train-logloss:0.26561	val-logloss:0.14991
[40]	train-logloss:0.05308	val-logloss:0.01052
[80]	train-logloss:0.03757	val-logloss:0.00881
[120]	train-logloss:0.03002	val-logloss:0.00785
[160]	train-logloss:0.02533	val-logloss:0.00717
[200]	train-logloss:0.02246	val-logloss:0.00672
[240]	train-logloss:0.02012	val-logloss:0.00650
[280]	train-logloss:0.01843	val-logloss:0.00616
[320]	train-logloss:0.01695	val-logloss:0.00609
[360]	train-logloss:0.01569	val-logloss:0.00603
[400]	train-logloss:0.01458	val-logloss:0.00608
[440]	train-logloss:0.01353	val-logloss:0.00611
[454]	train-logloss:0.01327	val-logloss:0.00608
best score 0.00600 at iteration 355
CPU times: user 7h 33min 29s, sys: 24min 54s, total: 7h 58min 23s
Wall time: 36min


In [7]:
%%time
y_true = val[label].tolist()
y_pred = model.predict(data=dval, iteration_range=(0, model.best_iteration+1))
auc = roc_auc_score(y_true, y_pred, average="macro")
print(f"auc={auc:.4f}")
print(f"y_pred={y_pred.shape}\n{y_pred[:5]}")

auc=0.9967
y_pred=(10492,)
[0.3766081  0.00339196 0.00248083 0.54953146 0.88870656]
CPU times: user 1.5 s, sys: 716 ms, total: 2.21 s
Wall time: 169 ms


In [8]:
%%time
scores = model.get_score(importance_type="gain")
assert len(scores)!=0
rows = []
for feature, score in scores.items():
    rows.append({'importance': score, 'feature': feature})
idf = pd.DataFrame.from_records(rows)
idf = idf.sort_values(["importance"], ascending=False, ignore_index=True)
fp = f"{job_dir}/importance.csv"
idf.to_csv(fp, index=True)
print(f"Saved {fp}")
idf.T.head()

Saved models/xgb/20240122_184023/importance.csv
CPU times: user 87.3 ms, sys: 86.6 ms, total: 174 ms
Wall time: 13.6 ms


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730
importance,433.323395,385.792328,380.499329,380.087921,361.889893,343.634277,343.081482,341.939484,319.657501,313.558594,312.684082,310.210541,304.799835,262.618195,233.207977,231.969193,230.78627,222.091797,207.405396,206.83757,204.23175,202.666595,200.029877,199.18042,198.433304,198.401779,189.276917,182.91597,170.924911,168.475098,167.760818,166.377121,165.27356,164.838852,162.64386,156.134125,153.912292,150.026962,149.817657,143.674301,140.578644,139.41983,138.213104,129.558456,125.866211,125.331383,125.046547,121.192047,119.348114,118.610069,117.920105,117.469292,114.029762,108.93026,108.695351,108.600037,108.339996,107.097801,107.033691,106.885529,104.560791,102.389465,102.194031,98.94503,97.796944,97.085754,93.614357,93.329773,90.678322,89.621902,88.374352,86.30201,82.872757,81.75087,80.869972,79.875946,79.604172,78.028732,77.060249,76.200775,75.807159,75.663818,74.370018,72.410812,72.282677,70.692719,70.358032,70.158569,69.419052,69.108536,69.100128,68.839737,68.772453,66.60788,65.971977,65.567268,65.324738,64.331306,64.243698,64.158653,63.939556,62.915257,62.389462,61.880489,61.703964,61.571636,61.478851,61.466423,60.63414,60.209877,59.926155,59.062714,59.03

In [9]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:38:53.990101
